In [2]:
import torch
torch.set_default_tensor_type(torch.FloatTensor)
from torch.autograd import Variable
from torch.nn import functional as F
from torchmetrics import Accuracy, Recall, Precision, Specificity, ConfusionMatrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_recall_curve, average_precision_score,roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
from collections import Counter
import random
import math
import torch.optim as optim
from tabulate import tabulate
from ray import tune

from fractions import Fraction

import models
import data_generation
import evaluation

import os

# import ray
# ray.init(num_cpus=30)


## linear-homogeneity

In [3]:
#linear-homo
beta_1=np.append(np.array([4,4,-8,-4,6,-4,8,4,-6,-8]),np.zeros(90))
beta_2=np.append(np.array([-2,2,4,-2,3,2,-4,-2,-3,4]),np.zeros(90))
beta_3=np.append(np.array([-3,-3,6,-3,-4.5,3,-6,-3,4.5,6]),np.zeros(90))

beta_1=beta_1.reshape(-1,1)
beta_2=beta_2.reshape(-1,1)
beta_3=beta_3.reshape(-1,1)

n1=200
n2=200
n3=200
p=100

## M1-s1

In [3]:
method="M1"
corval=0
prior=[0,1,2,3,4]
vs = np.zeros((1,100))

In [5]:
for i in range(1):
    data_generation.setup_seed(i)
    x1,y1=data_generation.generate_data_linear(corval,beta_1,n1,p)
    x2,y2=data_generation.generate_data_linear(corval,beta_2,n2,p)
    x3,y3=data_generation.generate_data_linear(corval,beta_3,n3,p)
    
    train_x1, test_x1, train_y1, test_y1= train_test_split(x1,y1, test_size=0.2, random_state=1)
    train_x2, test_x2, train_y2, test_y2= train_test_split(x2,y2, test_size=0.2, random_state=1)
    train_x3, test_x3, train_y3, test_y3= train_test_split(x3,y3, test_size=0.2, random_state=1)
    
     # switch training set
    inputs_1,targets_1 = train_x1,train_y1
    inputs_2,targets_2 = train_x2,train_y2
    inputs_3,targets_3 = train_x3,train_y3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3
    
################## first step######################################  
    prior=[0,1,2,3,4]
# choose lambda1,lambda2 by ray tune
    config={
            "lr":tune.choice([0.6,0.7,0.8]),
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "lambda2": tune.choice([0.001,0.002,0.003]),
            "ga":tune.choice([0.3,0.4,0.5]),
            "inputs_1":train_x1,
            "inputs_2":train_x2,
            "inputs_3":train_x3,
           "targets_1":train_y1,
           "targets_2":train_y2,
           "targets_3":train_y3,
            "prior":prior
        }

    result = tune.run(
            models.train_model_main_prior,
            config=config,
            num_samples=1)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    y_prior_1,y_prior_2,y_prior_3=models.main_prior_model(final_lambda["lambda1"],final_lambda["lambda2"],final_lambda["lr"],final_lambda["ga"],
                           inputs_1,inputs_2,inputs_3,targets_1,targets_2,targets_3,prior) 
    
################## third step##################
    no_prior=[]

    config={
                "lambda1": tune.choice([0.03,0.04,0.05]),
                "lambda2": tune.choice([0.001,0.002,0.003]),
                 "lr": tune.choice([0.7,0.8,0.9]),
                "eta": tune.choice([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]),
                 "ga":tune.choice([0.3]),
                "inputs_1":train_x1,
                "inputs_2":train_x2,
                "inputs_3":train_x3,
                "train_y1":train_y1,
                "train_y2":train_y2,
                "train_y3":train_y3,
                "y_prior_1":y_prior_1,
                "y_prior_2":y_prior_2,
                "y_prior_3":y_prior_3,
                "prior":no_prior
            }

    result = tune.run(
            models.train_model_main_final,  
            config=config,
            num_samples=1)

################## fourth step##################

    #no prior
    no_prior=[]
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_sum_tGM',mode='max')
    prediction,target,model_1_weight,model_2_weight,model_3_weight=models.main_final_model(final_parameter["lambda1"],final_parameter["lambda2"],final_parameter["lr"],final_parameter["ga"],final_parameter["eta"],
                       inputs_1,inputs_2,inputs_3,train_y1,train_y2,train_y3,y_prior_1,y_prior_2,y_prior_3,
                       test_inputs_1,test_inputs_2,test_inputs_3,test_targets_1,test_targets_2,test_targets_3,no_prior)
    
####################################   prediction ####################################      

    final_matrix=confusion_matrix(target,prediction)
    TPR=recall_score(target,prediction)#TPR
    TNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
    GM = math.sqrt(TPR*TNR)
    accuracy=accuracy_score(target,prediction)
    precision=precision_score(target,prediction)
    f1=f1_score(target,prediction)
    
    
####################################   variable selection ####################################     
    index,TP,FP,FN,TN=evaluation.variable_selection_homo_inte(0.12,model_1_weight,model_2_weight,model_3_weight,beta_1,p)

    vsSEN=TP/(TP+FN)
    vsSPE=(TN/(TN+FP))
    vsGM=math.sqrt(vsSEN*vsSPE)
    vsMR=(FP+FN)/(TP+FN+TN+FP)
    vsCCR=1-vsMR
    
    vs[:,list(index)]+=1
    
    print(i)
    
    log_file_name = "{}.csv".format("PPIN")
    log_file_dir = "./results/simulation/"
    log_file_path = os.path.join(log_file_dir, log_file_name)

    with open(log_file_path, "a") as f:
        logs= "{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(method, corval, len(prior),vsGM,vsSEN,vsSPE,vsMR,vsCCR,GM,TPR,TNR,accuracy,precision,f1)
        f.writelines(logs)
    

2022-11-30 14:38:05,519	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2022-11-30 14:38:06,499	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,ga,lambda1,lambda2,lr,iter,total time (s),my_test_BCE_loss
train_model_main_prior_8c450_00000,TERMINATED,10.224.0.12:11286,0.3,0.03,0.003,0.8,1,7.5794,1.34117


(raylet) [2022-11-30 14:38:15,403 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 265498624; capacity: 29394726912. Object creation will fail if spilling is required.


Result for train_model_main_prior_8c450_00000:
  date: 2022-11-30_14-38-16
  done: false
  experiment_id: 3b3a54cfd850477298bfa1026540ccf5
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.3411664724349976
  node_ip: 10.224.0.12
  pid: 11286
  time_since_restore: 7.579400539398193
  time_this_iter_s: 7.579400539398193
  time_total_s: 7.579400539398193
  timestamp: 1669790296
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 8c450_00000
  warmup_time: 0.006536722183227539
  
Result for train_model_main_prior_8c450_00000:
  date: 2022-11-30_14-38-16
  done: true
  experiment_id: 3b3a54cfd850477298bfa1026540ccf5
  experiment_tag: 0_ga=0.3000,lambda1=0.0300,lambda2=0.0030,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 1.3411664724349976
  node_ip: 10.224.0.12
  pid: 11286
  time_since_restore: 7.579400539398193
  time_this_iter_s: 7.579400539398193
  time_total_s: 7.579400539398193
  timestamp: 16

2022-11-30 14:38:16,602	INFO tune.py:758 -- Total run time: 10.11 seconds (9.94 seconds for the tuning loop).


...

0


(raylet) [2022-11-30 14:38:35,420 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263782400; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:38:45,428 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263782400; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:38:55,438 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263774208; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:39:05,456 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263741440; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-30 14:43:45,721 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263274496; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:43:55,730 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263274496; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:44:05,740 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263249920; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:44:15,750 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 263241728; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-30 14:48:56,009 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 259846144; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:49:06,018 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 259813376; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:49:16,028 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 259723264; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:49:26,037 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 259719168; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-30 14:54:06,291 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 259219456; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:54:16,298 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 259219456; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:54:26,305 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 259219456; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 14:54:36,312 E 10376 10394] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_14-38-03_471414_10262 is over 95% full, available space: 259215360; capacity: 29394726912. Object creation will fail i

## M2

In [4]:
method="M2"
corval=0
prior=[]
vs = np.zeros((1,100))

In [7]:
for i in range(1):
    data_generation.setup_seed(i)
    x1,y1=data_generation.generate_data_linear(corval,beta_1,n1,p)
    x2,y2=data_generation.generate_data_linear(corval,beta_2,n2,p)
    x3,y3=data_generation.generate_data_linear(corval,beta_3,n3,p)
    
    train_x1, test_x1, train_y1, test_y1= train_test_split(x1,y1, test_size=0.2, random_state=1)
    train_x2, test_x2, train_y2, test_y2= train_test_split(x2,y2, test_size=0.2, random_state=1)
    train_x3, test_x3, train_y3, test_y3= train_test_split(x3,y3, test_size=0.2, random_state=1)
    
     # switch training set
    inputs_1,targets_1 = train_x1,train_y1
    inputs_2,targets_2 = train_x2,train_y2
    inputs_3,targets_3 = train_x3,train_y3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3
    
################## first step######################################  
    no_prior=[]

    config={
                "lambda1": tune.choice([0.03,0.04,0.05]),
                "lambda2": tune.choice([0.001,0.002,0.003]),
                 "lr": tune.choice([0.7,0.8,0.9]),
                 "ga":tune.choice([0.3]),
                "inputs_1":train_x1,
                "inputs_2":train_x2,
                "inputs_3":train_x3,
                "targets_1":train_y1,
                "targets_2":train_y2,
                "targets_3":train_y3,
                "prior":no_prior
            }

    result = tune.run(
            models.train_model_no_prior_final,  
            config=config,
            num_samples=1)

################## fourth step##################

    #no prior
    no_prior=[]
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_sum_tGM',mode='max')
    prediction,target,model_1_weight,model_2_weight,model_3_weight=models.no_prior_final_model(final_parameter["lambda1"],final_parameter["lambda2"],final_parameter["lr"],final_parameter["ga"],
                       inputs_1,inputs_2,inputs_3,train_y1,train_y2,train_y3,
                       test_inputs_1,test_inputs_2,test_inputs_3,test_targets_1,test_targets_2,test_targets_3,no_prior)
    
####################################   prediction ####################################      

    final_matrix=confusion_matrix(target,prediction)
    TPR=recall_score(target,prediction)#TPR
    TNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
    GM = math.sqrt(TPR*TNR)
    accuracy=accuracy_score(target,prediction)
    precision=precision_score(target,prediction)
    f1=f1_score(target,prediction)
    
    
####################################   variable selection ####################################     
    index,TP,FP,FN,TN=evaluation.variable_selection_homo_inte(0.12,model_1_weight,model_2_weight,model_3_weight,beta_1,p)

    vsSEN=TP/(TP+FN)
    vsSPE=(TN/(TN+FP))
    vsGM=math.sqrt(vsSEN*vsSPE)
    vsMR=(FP+FN)/(TP+FN+TN+FP)
    vsCCR=1-vsMR
    
    vs[:,list(index)]+=1
    
    print(i)
    
    log_file_name = "{}.csv".format("PPIN")
    log_file_dir = "./results/simulation/"
    log_file_path = os.path.join(log_file_dir, log_file_name)

    with open(log_file_path, "a") as f:
        logs= "{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(method, corval, len(prior),vsGM,vsSEN,vsSPE,vsMR,vsCCR,GM,TPR,TNR,accuracy,precision,f1)
        f.writelines(logs)
    

Trial name,status,loc,ga,lambda1,lambda2,lr,iter,total time (s),my_test_sum_tGM
train_model_no_prior_final_45bbf_00000,TERMINATED,10.224.0.12:32010,0.3,0.05,0.001,0.9,1,8.75133,0.852142


(raylet) [2022-11-30 15:26:22,677 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 251998208; capacity: 29394726912. Object creation will fail if spilling is required.


Result for train_model_no_prior_final_45bbf_00000:
  date: 2022-11-30_15-26-25
  done: false
  experiment_id: 1f4d823521cf46a8b09ccfa0c1364f48
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.8521421657988256
  node_ip: 10.224.0.12
  pid: 32010
  time_since_restore: 8.751327753067017
  time_this_iter_s: 8.751327753067017
  time_total_s: 8.751327753067017
  timestamp: 1669793185
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 45bbf_00000
  warmup_time: 0.006920576095581055
  
Result for train_model_no_prior_final_45bbf_00000:
  date: 2022-11-30_15-26-25
  done: true
  experiment_id: 1f4d823521cf46a8b09ccfa0c1364f48
  experiment_tag: 0_ga=0.3000,lambda1=0.0500,lambda2=0.0010,lr=0.9000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.8521421657988256
  node_ip: 10.224.0.12
  pid: 32010
  time_since_restore: 8.751327753067017
  time_this_iter_s: 8.751327753067017
  time_total_s: 8.751327753067017
  timesta

2022-11-30 15:26:25,882	INFO tune.py:758 -- Total run time: 11.24 seconds (11.08 seconds for the tuning loop).


0


...

(raylet) [2022-11-30 15:52:24,126 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 249208832; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 15:52:34,135 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 249208832; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 15:52:44,145 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 249200640; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 15:52:54,155 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 249196544; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-30 15:57:34,411 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 248745984; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 15:57:44,420 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 248741888; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 15:57:54,429 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 248737792; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 15:58:04,439 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 248737792; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-30 16:02:44,708 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 248274944; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 16:02:54,718 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 248270848; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 16:03:04,727 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 248270848; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 16:03:14,737 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 248242176; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-30 16:07:55,006 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 247926784; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 16:08:05,016 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 247922688; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 16:08:15,024 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 247771136; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 16:08:25,033 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 247767040; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-30 16:13:05,296 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 247455744; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 16:13:15,305 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 247431168; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 16:13:25,314 E 31028 31053] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_15-25-10_722358_30914 is over 95% full, available space: 247427072; capacity: 29394726912. Object creation will fail if spilling is required.


## M3-s1

In [3]:
method="M3"
corval=0
prior=[0,1,2,3,4]
vs = np.zeros((1,100))

In [4]:
for i in range(1):
    data_generation.setup_seed(i)
    
    x1,y1=data_generation.generate_data_linear(corval,beta_1,n1,p)
    x2,y2=data_generation.generate_data_linear(corval,beta_2,n2,p)
    x3,y3=data_generation.generate_data_linear(corval,beta_3,n3,p)
    
    x= torch.cat((x1,x2), dim=0)
    x= torch.cat((x,x3), dim=0)
    y= torch.cat((y1,y2),dim=0)
    y= torch.cat((y,y3), dim=0)
    
    train_x, test_x, train_y, test_y= train_test_split(x,y, test_size=0.2, random_state=1)
    

    # switching testing set
    test_inputs, test_targets = test_x,test_y
    
################## first step######################################  
    prior=[0,1,2,3,4]

    config={
            "lr":tune.choice([0.6,0.7,0.8]),
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "lambda2": tune.choice([0.001,0.002,0.003]),
            "ga":tune.choice([0.3,0.4,0.5]),
            "inputs":train_x,
           "targets":train_y,
            "prior":prior
        }

    result = tune.run(
            models.train_model_pDFS_prior,  
            config=config,
            num_samples=1)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    y_prior=models.pDFS_prior_model(final_lambda["lambda1"],final_lambda["lambda2"],final_lambda["lr"],final_lambda["ga"],
                           train_x,train_y,prior) 

################## third step##################

    #no prior
    no_prior=[]

    config={
                "lambda1": tune.choice([0.03,0.04,0.05]),
                "lambda2": tune.choice([0.001,0.002,0.003]),
                 "lr": tune.choice([0.7,0.8,0.9]),
                "eta": tune.choice([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]),
                 "ga":tune.choice([0.3]),
                "inputs":train_x,
                "train_y":train_y,
                "y_prior":y_prior,
                "prior":no_prior
            }

    result = tune.run(
            models.train_model_pDFS_final,  
            config=config,
            num_samples=1)

################## fourth step##################

    #no prior
    no_prior=[]
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_sum_tGM',mode='max')
    prediction,target,model_weight=models.pDFS_final_model(final_parameter["lambda1"],final_parameter["lambda2"],final_parameter["lr"],final_parameter["ga"],final_parameter["eta"],
                       train_x,train_y,y_prior,
                       test_inputs,test_targets,no_prior)
    
####################################   prediction ####################################   
    final_matrix=confusion_matrix(target,prediction)
    TPR=recall_score(target,prediction)#TPR
    TNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
    GM=math.sqrt(TPR*TNR)
    accuracy=accuracy_score(target,prediction)
    precision=precision_score(target,prediction)
    f1=f1_score(target,prediction)
    
####################################   variable selection ####################################     
    index,TP,FP,FN,TN=evaluation.variable_selection_homo_DFS(0.12,model_weight,beta_1,p)
    
    vsSEN=TP/(TP+FN)
    vsSPE=TN/(TN+FP)
    vsGM=math.sqrt(vsSEN*vsSPE)
    vsMR=FP+FN/(TP+FN+TN+FP)
    vsCCR=1-vsMR
    
    vs[:,list(index)]+=1
    
    print(i)
    
    log_file_name = "{}.csv".format("PPIN")
    log_file_dir = "./results/simulation/"
    log_file_path = os.path.join(log_file_dir, log_file_name)

    with open(log_file_path, "a") as f:
        logs= "{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(method, corval, len(prior),vsGM,vsSEN,vsSPE,vsMR,vsCCR,GM,TPR,TNR,accuracy,precision,f1)
        f.writelines(logs)

2022-11-30 17:44:22,360	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2022-11-30 17:44:23,128	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,ga,lambda1,lambda2,lr,iter,total time (s),my_test_BCE_loss
train_model_pDFS_prior_920f4_00000,TERMINATED,10.224.0.12:31066,0.3,0.03,0.003,0.8,1,4.51445,0.713986


Result for train_model_pDFS_prior_920f4_00000:
  date: 2022-11-30_17-44-29
  done: false
  experiment_id: b1af9622d2c94e34bd20a42e79e0427e
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7139863252639771
  node_ip: 10.224.0.12
  pid: 31066
  time_since_restore: 4.514451742172241
  time_this_iter_s: 4.514451742172241
  time_total_s: 4.514451742172241
  timestamp: 1669801469
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 920f4_00000
  warmup_time: 0.00624537467956543
  
Result for train_model_pDFS_prior_920f4_00000:
  date: 2022-11-30_17-44-29
  done: true
  experiment_id: b1af9622d2c94e34bd20a42e79e0427e
  experiment_tag: 0_ga=0.3000,lambda1=0.0300,lambda2=0.0030,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.7139863252639771
  node_ip: 10.224.0.12
  pid: 31066
  time_since_restore: 4.514451742172241
  time_this_iter_s: 4.514451742172241
  time_total_s: 4.514451742172241
  timestamp: 166

2022-11-30 17:44:30,148	INFO tune.py:758 -- Total run time: 7.02 seconds (6.88 seconds for the tuning loop).


Trial name,status,loc,eta,ga,lambda1,lambda2,lr,iter,total time (s),my_test_sum_tGM
train_model_pDFS_final_96e62_00000,TERMINATED,10.224.0.12:32096,0,0.3,0.05,0.001,0.7,1,4.97191,0.153075


(raylet) [2022-11-30 17:44:32,248 E 30156 30181] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_17-44-20_330050_30042 is over 95% full, available space: 220852224; capacity: 29394726912. Object creation will fail if spilling is required.


Result for train_model_pDFS_final_96e62_00000:
  date: 2022-11-30_17-44-38
  done: false
  experiment_id: eb3a918f90f340dda518a5d9986a9664
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.15307525638391256
  node_ip: 10.224.0.12
  pid: 32096
  time_since_restore: 4.971913576126099
  time_this_iter_s: 4.971913576126099
  time_total_s: 4.971913576126099
  timestamp: 1669801478
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 96e62_00000
  warmup_time: 0.0067138671875
  
Result for train_model_pDFS_final_96e62_00000:
  date: 2022-11-30_17-44-38
  done: true
  experiment_id: eb3a918f90f340dda518a5d9986a9664
  experiment_tag: 0_eta=0,ga=0.3000,lambda1=0.0500,lambda2=0.0010,lr=0.7000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.15307525638391256
  node_ip: 10.224.0.12
  pid: 32096
  time_since_restore: 4.971913576126099
  time_this_iter_s: 4.971913576126099
  time_total_s: 4.971913576126099
  timestamp: 1

2022-11-30 17:44:38,673	INFO tune.py:758 -- Total run time: 7.42 seconds (7.29 seconds for the tuning loop).


0
{'lambda1': 0.05, 'lambda2': 0.001, 'lr': 0.7, 'eta': 0, 'ga': 0.3, 'inputs': tensor([[ 0.4128,  0.0746,  1.4729,  ..., -0.8821,  1.7233, -0.5655],
        [-1.1291, -1.2914,  1.1162,  ..., -0.1507,  2.5223,  0.8216],
        [-1.7496,  0.5991, -0.5499,  ...,  0.7662, -0.7397, -1.2588],
        ...,
        [-0.0887,  1.5128,  0.5737,  ..., -1.1701,  0.4273, -0.4869],
        [-0.5543,  0.0906,  0.0505,  ..., -0.3244,  1.2577,  0.9913],
        [-0.4148,  1.7565,  0.6480,  ..., -0.7308, -1.3102,  1.0791]]), 'train_y': tensor([0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1.,
        1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
        0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1.,
        1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0.,
        0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1.,
        1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1.

/home/rd/anaconda3/envs/envJK/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
(raylet) [2022-11-30 17:44:42,255 E 30156 30181] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_17-44-20_330050_30042 is over 95% full, available space: 220422144; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 17:44:52,264 E 30156 30181] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_17-44-20_330050_30042 is over 95% full, available space: 220422144; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 17:45:02,273 E 30156 30181] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_17-44-20_330050_30042 is over 95% full, available space: 2204057

## M4_s1

In [ ]:
method="M4-1"
corval=0
prior=[0,1,2,3,4]
vs = np.zeros((1,100))

In [ ]:
for i in range(1):
    data_generation.setup_seed(i)
    
    x1,y1=data_generation.generate_data_linear(corval,beta_1,n1,p)

   
    train_x, test_x, train_y, test_y= train_test_split(x1,y1, test_size=0.2, random_state=1)
    

    # switching testing set
    test_inputs, test_targets = test_x,test_y
    
################## first step######################################  
    prior=[0,1,2,3,4]

    config={
            "lr":tune.choice([0.6,0.7,0.8]),
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "lambda2": tune.choice([0.001,0.002,0.003]),
            "ga":tune.choice([0.3,0.4,0.5]),
            "inputs":train_x,
           "targets":train_y,
            "prior":prior
        }

    result = tune.run(
            models.train_model_pDFS_prior,  
            config=config,
            num_samples=1)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    y_prior=models.pDFS_prior_model(final_lambda["lambda1"],final_lambda["lambda2"],final_lambda["lr"],final_lambda["ga"],
                           train_x,train_y,prior) 

################## third step##################

    #no prior
    no_prior=[]

    config={
                "lambda1": tune.choice([0.03,0.04,0.05]),
                "lambda2": tune.choice([0.001,0.002,0.003]),
                 "lr": tune.choice([0.7,0.8,0.9]),
                "eta": tune.choice([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]),
                 "ga":tune.choice([0.3]),
                "inputs":train_x,
                "train_y":train_y,
                "y_prior":y_prior,
                "prior":no_prior
            }

    result = tune.run(
            models.train_model_pDFS_final,  
            config=config,
            num_samples=1)

################## fourth step##################

    #no prior
    no_prior=[]
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_sum_tGM',mode='max')
    prediction,target,model_weight=models.pDFS_final_model(final_parameter["lambda1"],final_parameter["lambda2"],final_parameter["lr"],final_parameter["ga"],final_parameter["eta"],
                       train_x,train_y,y_prior,
                       test_inputs,test_targets,no_prior)
    
####################################   prediction ####################################   
    final_matrix=confusion_matrix(target,prediction)
    TPR=recall_score(target,prediction)#TPR
    TNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
    GM=math.sqrt(TPR*TNR)
    accuracy=accuracy_score(target,prediction)
    precision=precision_score(target,prediction)
    f1=f1_score(target,prediction)
    
####################################   variable selection ####################################     
    index,TP,FP,FN,TN=evaluation.variable_selection_homo_DFS(0.12,model_weight,beta_1,p)
    
    vsSEN=TP/(TP+FN)
    vsSPE=TN/(TN+FP)
    vsGM=math.sqrt(vsSEN*vsSPE)
    vsMR=FP+FN/(TP+FN+TN+FP)
    vsCCR=1-vsMR
    
    vs[:,list(index)]+=1
    
    print(i)
    
    log_file_name = "{}.csv".format("PPIN")
    log_file_dir = "./results/simulation/"
    log_file_path = os.path.join(log_file_dir, log_file_name)

    with open(log_file_path, "a") as f:
        logs= "{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(method, corval, len(prior),vsGM,vsSEN,vsSPE,vsMR,vsCCR,GM,TPR,TNR,accuracy,precision,f1)
        f.writelines(logs)

In [ ]:
method="M4-2"
corval=0
prior=[0,1,2,3,4]
vs = np.zeros((1,100))

In [ ]:
for i in range(1):
    data_generation.setup_seed(i)
    
    x2,y2=data_generation.generate_data_linear(corval,beta_2,n2,p)

   
    train_x, test_x, train_y, test_y= train_test_split(x2,y2, test_size=0.2, random_state=1)
    

    # switching testing set
    test_inputs, test_targets = test_x,test_y
    
################## first step######################################  
    prior=[0,1,2,3,4]

    config={
            "lr":tune.choice([0.6,0.7,0.8]),
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "lambda2": tune.choice([0.001,0.002,0.003]),
            "ga":tune.choice([0.3,0.4,0.5]),
            "inputs":train_x,
           "targets":train_y,
            "prior":prior
        }

    result = tune.run(
            models.train_model_pDFS_prior,  
            config=config,
            num_samples=1)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    y_prior=models.pDFS_prior_model(final_lambda["lambda1"],final_lambda["lambda2"],final_lambda["lr"],final_lambda["ga"],
                           train_x,train_y,prior) 

################## third step##################

    #no prior
    no_prior=[]

    config={
                "lambda1": tune.choice([0.03,0.04,0.05]),
                "lambda2": tune.choice([0.001,0.002,0.003]),
                 "lr": tune.choice([0.7,0.8,0.9]),
                "eta": tune.choice([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]),
                 "ga":tune.choice([0.3]),
                "inputs":train_x,
                "train_y":train_y,
                "y_prior":y_prior,
                "prior":no_prior
            }

    result = tune.run(
            models.train_model_pDFS_final,  
            config=config,
            num_samples=1)

################## fourth step##################

    #no prior
    no_prior=[]
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_sum_tGM',mode='max')
    prediction,target,model_weight=models.pDFS_final_model(final_parameter["lambda1"],final_parameter["lambda2"],final_parameter["lr"],final_parameter["ga"],final_parameter["eta"],
                       train_x,train_y,y_prior,
                       test_inputs,test_targets,no_prior)
    
####################################   prediction ####################################   
    final_matrix=confusion_matrix(target,prediction)
    TPR=recall_score(target,prediction)#TPR
    TNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
    GM=math.sqrt(TPR*TNR)
    accuracy=accuracy_score(target,prediction)
    precision=precision_score(target,prediction)
    f1=f1_score(target,prediction)
    
####################################   variable selection ####################################     
    index,TP,FP,FN,TN=evaluation.variable_selection_homo_DFS(0.12,model_weight,beta_2,p)
    
    vsSEN=TP/(TP+FN)
    vsSPE=TN/(TN+FP)
    vsGM=math.sqrt(vsSEN*vsSPE)
    vsMR=FP+FN/(TP+FN+TN+FP)
    vsCCR=1-vsMR
    
    vs[:,list(index)]+=1
    
    print(i)
    
    log_file_name = "{}.csv".format("PPIN")
    log_file_dir = "./results/simulation/"
    log_file_path = os.path.join(log_file_dir, log_file_name)

    with open(log_file_path, "a") as f:
        logs= "{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(method, corval, len(prior),vsGM,vsSEN,vsSPE,vsMR,vsCCR,GM,TPR,TNR,accuracy,precision,f1)
        f.writelines(logs)

In [ ]:
method="M4-3"
corval=0
prior=[0,1,2,3,4]
vs = np.zeros((1,100))

In [ ]:
for i in range(1):
    data_generation.setup_seed(i)
    
    x3,y3=data_generation.generate_data_linear(corval,beta_3,n3,p)

   
    train_x, test_x, train_y, test_y= train_test_split(x3,y3, test_size=0.2, random_state=1)
    

    # switching testing set
    test_inputs, test_targets = test_x,test_y
    
################## first step######################################  
    prior=[0,1,2,3,4]

    config={
            "lr":tune.choice([0.6,0.7,0.8]),
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "lambda2": tune.choice([0.001,0.002,0.003]),
            "ga":tune.choice([0.3,0.4,0.5]),
            "inputs":train_x,
           "targets":train_y,
            "prior":prior
        }

    result = tune.run(
            models.train_model_pDFS_prior,  
            config=config,
            num_samples=1)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    y_prior=models.pDFS_prior_model(final_lambda["lambda1"],final_lambda["lambda2"],final_lambda["lr"],final_lambda["ga"],
                           train_x,train_y,prior) 

################## third step##################

    #no prior
    no_prior=[]

    config={
                "lambda1": tune.choice([0.03,0.04,0.05]),
                "lambda2": tune.choice([0.001,0.002,0.003]),
                 "lr": tune.choice([0.7,0.8,0.9]),
                "eta": tune.choice([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]),
                 "ga":tune.choice([0.3]),
                "inputs":train_x,
                "train_y":train_y,
                "y_prior":y_prior,
                "prior":no_prior
            }

    result = tune.run(
            models.train_model_pDFS_final,  
            config=config,
            num_samples=1)

################## fourth step##################

    #no prior
    no_prior=[]
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_sum_tGM',mode='max')
    prediction,target,model_weight=models.pDFS_final_model(final_parameter["lambda1"],final_parameter["lambda2"],final_parameter["lr"],final_parameter["ga"],final_parameter["eta"],
                       train_x,train_y,y_prior,
                       test_inputs,test_targets,no_prior)
    
####################################   prediction ####################################   
    final_matrix=confusion_matrix(target,prediction)
    TPR=recall_score(target,prediction)#TPR
    TNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
    GM=math.sqrt(TPR*TNR)
    accuracy=accuracy_score(target,prediction)
    precision=precision_score(target,prediction)
    f1=f1_score(target,prediction)
    
####################################   variable selection ####################################     
    index,TP,FP,FN,TN=evaluation.variable_selection_homo_DFS(0.12,model_weight,beta_2,p)
    
    vsSEN=TP/(TP+FN)
    vsSPE=TN/(TN+FP)
    vsGM=math.sqrt(vsSEN*vsSPE)
    vsMR=FP+FN/(TP+FN+TN+FP)
    vsCCR=1-vsMR
    
    vs[:,list(index)]+=1
    
    print(i)
    
    log_file_name = "{}.csv".format("PPIN")
    log_file_dir = "./results/simulation/"
    log_file_path = os.path.join(log_file_dir, log_file_name)

    with open(log_file_path, "a") as f:
        logs= "{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(method, corval, len(prior),vsGM,vsSEN,vsSPE,vsMR,vsCCR,GM,TPR,TNR,accuracy,precision,f1)
        f.writelines(logs)

## M5-s1

In [4]:
method="M5"
corval=0
prior=[0,1,2,3,4]
vs = np.zeros((1,100))

In [5]:
for i in range(1):
    data_generation.setup_seed(i)
    x1,y1=data_generation.generate_data_linear(corval,beta_1,n1,p)
    x2,y2=data_generation.generate_data_linear(corval,beta_2,n2,p)
    x3,y3=data_generation.generate_data_linear(corval,beta_3,n3,p)
    
    train_x1, test_x1, train_y1, test_y1= train_test_split(x1,y1, test_size=0.2, random_state=1)
    train_x2, test_x2, train_y2, test_y2= train_test_split(x2,y2, test_size=0.2, random_state=1)
    train_x3, test_x3, train_y3, test_y3= train_test_split(x3,y3, test_size=0.2, random_state=1)
    
     # switch training set
    inputs_1,targets_1 = train_x1,train_y1
    inputs_2,targets_2 = train_x2,train_y2
    inputs_3,targets_3 = train_x3,train_y3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3
    
################## first step######################################  
    prior=[0,1,2,3,4]
# choose lambda1,lambda2 by ray tune
    config={
            "lr":tune.choice([0.6,0.7,0.8]),
            "lambda1": tune.choice([0.03,0.04,0.05]),
            "ga":tune.choice([0.3,0.4,0.5]),
            "inputs_1":train_x1,
            "inputs_2":train_x2,
            "inputs_3":train_x3,
           "targets_1":train_y1,
           "targets_2":train_y2,
           "targets_3":train_y3,
            "prior":prior
        }

    result = tune.run(
            models.train_model_IA_prior,
            config=config,
            num_samples=1)
    
################## second step########################################    

    final_lambda=result.get_best_config('my_test_BCE_loss',mode='min')
    y_prior_1,y_prior_2,y_prior_3=models.IA_prior_model(final_lambda["lambda1"],final_lambda["lr"],final_lambda["ga"],
                           inputs_1,inputs_2,inputs_3,targets_1,targets_2,targets_3,prior) 
    
################## third step##################
    no_prior=[]

    config={
                "lambda1": tune.choice([0.03,0.04,0.05]),
                 "lr": tune.choice([0.7,0.8,0.9]),
                "eta": tune.choice([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]),
                 "ga":tune.choice([0.3]),
                "inputs_1":train_x1,
                "inputs_2":train_x2,
                "inputs_3":train_x3,
                "train_y1":train_y1,
                "train_y2":train_y2,
                "train_y3":train_y3,
                "y_prior_1":y_prior_1,
                "y_prior_2":y_prior_2,
                "y_prior_3":y_prior_3,
                "prior":no_prior
            }

    result = tune.run(
            models.train_model_IA_final,  
            config=config,
            num_samples=1)

################## fourth step##################

    #no prior
    no_prior=[]
    #0.4,0.4
    # set hyperparamters
    final_parameter=result.get_best_config('my_test_sum_tGM',mode='max')
    prediction,target,model_1_weight,model_2_weight,model_3_weight=models.IA_final_model(final_parameter["lambda1"],final_parameter["lr"],final_parameter["ga"],final_parameter["eta"],
                       inputs_1,inputs_2,inputs_3,train_y1,train_y2,train_y3,y_prior_1,y_prior_2,y_prior_3,
                       test_inputs_1,test_inputs_2,test_inputs_3,test_targets_1,test_targets_2,test_targets_3,no_prior)
    
####################################   prediction ####################################      

    final_matrix=confusion_matrix(target,prediction)
    TPR=recall_score(target,prediction)#TPR
    TNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
    GM = math.sqrt(TPR*TNR)
    accuracy=accuracy_score(target,prediction)
    precision=precision_score(target,prediction)
    f1=f1_score(target,prediction)
    
    
####################################   variable selection ####################################     
    index,TP,FP,FN,TN=evaluation.variable_selection_homo_inte(0.12,model_1_weight,model_2_weight,model_3_weight,beta_1,p)

    vsSEN=TP/(TP+FN)
    vsSPE=(TN/(TN+FP))
    vsGM=math.sqrt(vsSEN*vsSPE)
    vsMR=(FP+FN)/(TP+FN+TN+FP)
    vsCCR=1-vsMR
    
    vs[:,list(index)]+=1
    
    print(i)
    
    log_file_name = "{}.csv".format("PPIN")
    log_file_dir = "./results/simulation/"
    log_file_path = os.path.join(log_file_dir, log_file_name)

    with open(log_file_path, "a") as f:
        logs= "{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(method, corval, len(prior),vsGM,vsSEN,vsSPE,vsMR,vsCCR,GM,TPR,TNR,accuracy,precision,f1)
        f.writelines(logs)
    

2022-11-30 18:22:39,360	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2022-11-30 18:22:40,150	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,ga,lambda1,lr,iter,total time (s),my_test_BCE_loss
train_model_IA_prior_eb30f_00000,TERMINATED,10.224.0.12:14593,0.5,0.03,0.8,1,4.81648,0.594329


Result for train_model_IA_prior_eb30f_00000:
  date: 2022-11-30_18-22-47
  done: false
  experiment_id: 76196871bb82424fabcd822fcebcc5b7
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.5943285584449768
  node_ip: 10.224.0.12
  pid: 14593
  time_since_restore: 4.816476583480835
  time_this_iter_s: 4.816476583480835
  time_total_s: 4.816476583480835
  timestamp: 1669803767
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: eb30f_00000
  warmup_time: 0.0065724849700927734
  
Result for train_model_IA_prior_eb30f_00000:
  date: 2022-11-30_18-22-47
  done: true
  experiment_id: 76196871bb82424fabcd822fcebcc5b7
  experiment_tag: 0_ga=0.5000,lambda1=0.0300,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_BCE_loss: 0.5943285584449768
  node_ip: 10.224.0.12
  pid: 14593
  time_since_restore: 4.816476583480835
  time_this_iter_s: 4.816476583480835
  time_total_s: 4.816476583480835
  timestamp: 1669803767
  timeste

2022-11-30 18:22:47,462	INFO tune.py:758 -- Total run time: 7.31 seconds (7.16 seconds for the tuning loop).


Trial name,status,loc,eta,ga,lambda1,lr,iter,total time (s),my_test_sum_tGM
train_model_IA_final_f01be_00000,TERMINATED,10.224.0.12:14635,0,0.3,0.03,0.9,1,4.68032,0.929691


(raylet) [2022-11-30 18:22:49,245 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 241844224; capacity: 29394726912. Object creation will fail if spilling is required.


Result for train_model_IA_final_f01be_00000:
  date: 2022-11-30_18-22-55
  done: false
  experiment_id: f65de85fe82d482abeef6e3151fd354f
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9296913047420498
  node_ip: 10.224.0.12
  pid: 14635
  time_since_restore: 4.680323600769043
  time_this_iter_s: 4.680323600769043
  time_total_s: 4.680323600769043
  timestamp: 1669803775
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: f01be_00000
  warmup_time: 0.006989002227783203
  
Result for train_model_IA_final_f01be_00000:
  date: 2022-11-30_18-22-55
  done: true
  experiment_id: f65de85fe82d482abeef6e3151fd354f
  experiment_tag: 0_eta=0,ga=0.3000,lambda1=0.0300,lr=0.9000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.9296913047420498
  node_ip: 10.224.0.12
  pid: 14635
  time_since_restore: 4.680323600769043
  time_this_iter_s: 4.680323600769043
  time_total_s: 4.680323600769043
  timestamp: 1669803775
  time

2022-11-30 18:22:55,549	INFO tune.py:758 -- Total run time: 7.15 seconds (7.01 seconds for the tuning loop).


0


(raylet) [2022-11-30 18:22:59,253 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 241401856; capacity: 29394726912. Object creation will fail if spilling is required.


In [6]:
vsSEN

1.0

(raylet) [2022-11-30 18:23:09,261 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 241418240; capacity: 29394726912. Object creation will fail if spilling is required.


In [7]:
vs

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

(raylet) [2022-11-30 18:23:19,270 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 241418240; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:23:29,279 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 240988160; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:23:39,289 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 240975872; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:23:49,299 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 240951296; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-30 18:28:29,568 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 240504832; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:28:39,577 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 240480256; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:28:49,585 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 240472064; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:28:59,595 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 240467968; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-30 18:33:39,861 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 240132096; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:33:49,871 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 240037888; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:33:59,880 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 240037888; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:34:09,890 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 240037888; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-30 18:38:50,138 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 239529984; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:39:00,147 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 239529984; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:39:10,156 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 239525888; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:39:20,166 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 239521792; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-30 18:44:00,426 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 239067136; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:44:10,435 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 239063040; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:44:20,444 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 239063040; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:44:30,453 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 239063040; capacity: 29394726912. Object creation will fail i

(raylet) [2022-11-30 18:49:10,714 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 238600192; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-30 18:49:20,724 E 13688 13706] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-30_18-22-37_337502_13572 is over 95% full, available space: 238600192; capacity: 29394726912. Object creation will fail if spilling is required.
